In [1]:
# Importing necessary library- pandas
import pandas as pd

Event_data_raw = pd.read_csv('events.csv', header=0)
Event_data_raw['EventDateTime']= pd.to_datetime(Event_data_raw['EventDateTime'])

# Flag-2: Not considering first row as it contains old date for EventID: '223f3ffa-15e4-4b29-a328-488369b8c127'
Event_data = Event_data_raw[1:7]

# Flag-1: Dropping row wih NaN EventID
Event_data = Event_data.dropna(subset=['EventID'])

# Final Events table
Event_data

,EventID,PatientID,NHSClinicalCode1,NHSClinicalCode2,Postcode,EventDateTime,RegistrationDate,Name
1,3f56f414-5005-41ad-812c-5afa8107305a,cdc323c9-06e9-44a6-8fca-ba64be6d1d00,xx,YY,BT5,1999-02-03,01/01/1900,NaN
2,f557b132-6e09-4b8f-9762-e4724c00d007,27be53f8-b1a6-46b6-bb1f-8c0fbca50ac8,1,012,EN3,1999-02-04,01/01/1900,NaN
4,2971b0ea-df0d-4765-ab3c-956a40e1d969,df104bd6-52bd-4414-927a-77ea832e6872,66,444,NW2,1999-02-06,01/01/1900,NaN
5,d46d4595-b417-462b-aa7f-f02ffbf0b798,27ea2373-c482-4608-8a92-49605862961f,6,784,NW2,1999-02-07,01/01/1900,NaN
6,223f3ffa-15e4-4b29-a328-488369b8c127,cdc323c9-06e9-44a6-8fca-ba64be6d1d00,2,332,NW2,1999-02-08,01/01/1900,Jack Rowntree


In [2]:
Diagnosis_data_raw = pd.read_csv('diagnoses.csv', header=0)
Diagnosis_data_raw['DiagnosisDateTime']= pd.to_datetime(Diagnosis_data_raw['DiagnosisDateTime'])

# Flag-3: Dropping duplicate row
Diagnosis_data = Diagnosis_data_raw.drop_duplicates()

# Flag-1: Dropping row wih NaN Diagnosis values
Diagnosis_data = Diagnosis_data.dropna()

# Final Diagnosis table
Diagnosis_data

,EventID,Diagnosis,DiagnosisDateTime
0,223f3ffa-15e4-4b29-a328-488369b8c127,A,2020-02-01 13:00:00
1,3f56f414-5005-41ad-812c-5afa8107305a,B,2020-02-01 11:00:00
4,3f56f414-5005-41ad-812c-5afa8107305a,B,2020-02-01 18:00:00
5,f557b132-6e09-4b8f-9762-e4724c00d007,C,2020-02-01 15:00:00
6,ed7f9b79-e1e1-432e-a4f8-5282286dacac,A,2020-02-05 13:00:00
7,223f3ffa-15e4-4b29-a328-488369b8c127,B,2020-02-02 13:00:00
8,3f56f414-5005-41ad-812c-5afa8107305a,D,2020-03-01 13:00:00
9,f557b132-6e09-4b8f-9762-e4724c00d007,C,2020-12-01 13:00:00
10,3f56f414-5005-41ad-812c-5afa8107305a,B,1998-02-01 11:00:00


In [3]:
# Method to fetch index values for the most recent diagnosis after a particular event

def ref(df1, df2):
    index = []
    for i in df1.index:
        event_pre_id = df1['EventID'][i]
        index1 = []
        distance = []
        for j in df2.index:
            event_late_id = df2['EventID'][j]
            if event_late_id == event_pre_id:
                if (df2['DiagnosisDateTime'][j] > df1['EventDateTime'][i]):
                    dis = df2['DiagnosisDateTime'][j]-df1['EventDateTime'][i]
                    distance.append(dis)
                    index1.append(j)
        if len(distance) != 0:
            min_dis = distance.index(min(distance))
            index_need = index1[min_dis]
            index.append(index_need)
    return index

In [4]:
index = ref(Event_data,Diagnosis_data)
Event_data['MostRecentDiagnosis'] = 'No recent diagnosis'
for i in index:
    for j in Event_data.index:
        if Event_data['EventID'][j]== Diagnosis_data.loc[i, 'EventID']:
            Event_data['MostRecentDiagnosis'][j] = Diagnosis_data.loc[i, 'Diagnosis']
Event_data

<ipython-input-4-c20ad192c660>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Event_data['MostRecentDiagnosis'][j] = Diagnosis_data.loc[i, 'Diagnosis']


,EventID,PatientID,NHSClinicalCode1,NHSClinicalCode2,Postcode,EventDateTime,RegistrationDate,Name,MostRecentDiagnosis
1,3f56f414-5005-41ad-812c-5afa8107305a,cdc323c9-06e9-44a6-8fca-ba64be6d1d00,xx,YY,BT5,1999-02-03,01/01/1900,NaN,B
2,f557b132-6e09-4b8f-9762-e4724c00d007,27be53f8-b1a6-46b6-bb1f-8c0fbca50ac8,1,012,EN3,1999-02-04,01/01/1900,NaN,C
4,2971b0ea-df0d-4765-ab3c-956a40e1d969,df104bd6-52bd-4414-927a-77ea832e6872,66,444,NW2,1999-02-06,01/01/1900,NaN,No recent diagnosis
5,d46d4595-b417-462b-aa7f-f02ffbf0b798,27ea2373-c482-4608-8a92-49605862961f,6,784,NW2,1999-02-07,01/01/1900,NaN,No recent diagnosis
6,223f3ffa-15e4-4b29-a328-488369b8c127,cdc323c9-06e9-44a6-8fca-ba64be6d1d00,2,332,NW2,1999-02-08,01/01/1900,Jack Rowntree,A
